# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096825


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:07<01:42,  3.67s/it]

Rendering models:  10%|█         | 3/30 [00:07<01:13,  2.74s/it]

Rendering models:  13%|█▎        | 4/30 [00:08<00:53,  2.07s/it]

Rendering models:  17%|█▋        | 5/30 [00:09<00:40,  1.63s/it]

...

Rendering models:  87%|████████▋ | 26/30 [00:19<00:02,  1.59it/s]

Rendering models:  90%|█████████ | 27/30 [00:20<00:01,  1.72it/s]

Rendering models:  93%|█████████▎| 28/30 [00:21<00:01,  1.74it/s]

Rendering models: 100%|██████████| 30/30 [00:21<00:00,  2.14it/s]

not-logged-in-f45cae824435ec815f31    0.004783
jnarayanbvg                           0.007248
awright5                              0.001495
framosuw                              0.000610
TSM_Ginga                             0.000907
not-logged-in-c82b158ee9f8100a6b4e    0.003996
not-logged-in-323496bb8ce1ca59fd0a    0.002628
not-logged-in-5a8bb7be433678ade709    0.000696
marvinsfan                            0.003699
OmGupta                               0.000535
Marra                                 0.002263
not-logged-in-9a45911eb71184fe7c7f    0.000391
lumpofgiggles                         0.003482
not-logged-in-f8db0d00e76235ba4df0    0.000487
not-logged-in-2aaf71de46fc8e66d4f8    0.000369
databanana                            0.000378
pangeli5                              0.000928
Planetme                              0.000555
acapirala                             0.000370
alexmnelson18                         0.000647
Lavadude                              0.010556
Mr_DMan      

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())